# Backpropagation

Resumen de lo visto:

| Concepto                      | Exp                                                             | Ejemplo                         |
| ----------------------------- | ------------------------------------------------------------------------------------- | --------------------------------------------- |
| **Función de pérdida (loss)** | Mide discrepancia entre predicción y objetivo (≥ 0).                                  | `loss_fn = nn.MSELoss()`                      |
| **Gradiente**                 | Dirección de cambio más rápido del error respecto a cada peso.                        | `loss.backward()` calcula `W.grad`, `b.grad`. |
| **Learning rate**             | Tamaño del paso en la dirección del gradiente.                                        | `lr=0.05`                                     |
| **Optimización**              | Proceso de actualizar los pesos usando esos gradientes.                               | `optimizer.step()`                            |
| **SGD**                       | Mecanismo de actualización (no un modo en sí). Aleatoriedad solo si tú la introduces. | `torch.optim.SGD([W,b], lr=0.05)`             |
| **Backpropagation**           | Mecanismo automático que propaga gradientes hacia capas previas.                      | Ocurre dentro de `loss.backward()`.           |


Ahora veremos en profundidad el concepto de Backpropagation.

Backpropagation es simplemente “cómo se reparte la culpa del error”

Cuando la red se equivoca, ese error fluye hacia atrás capa por capa.
Cada peso recibe una parte proporcional de responsabilidad según cuánto haya contribuido al error.

#### 1.Cómo se propaga el error hacia atrás a través de las capas.

La regla de la cadena aplicada a funciones compuestas.
Cada capa transmite su “culpa” hacia la anterior.

#### 2. Por qué necesitamos las derivadas de las funciones de activación (ReLU, Sigmoid, etc.)

Son las “válvulas” que controlan cuánta información pasa hacia atrás.

Entender dónde pueden “matar” el gradiente (problema del vanishing gradient)

#### 3. Qué se guarda en la fase forward para poder hacer backward.

Los valores intermedios (z, a) que se necesitan para calcular derivadas.

### Empezamos.
Backpropagation empieza en la salida y empuja el error hacia atrás por la red.
Para entenderlo, basta una sola neurona: el flujo de gradientes será idéntico, solo que en redes profundas se encadena capa a capa.

In [1]:
# === 1. Preparación: Neurona simple con activación Sigmoid ===

import torch
import torch.nn.functional as F

torch.manual_seed(0)

# Datos de entrada (una sola muestra con 2 features)
x = torch.tensor([[2.0, 1.0]])   # shape (1, 2)

# Etiqueta verdadera (target)
y_true = torch.tensor([[1.0]])   # salida esperada

# Pesos y bias iniciales (entrenables)
W = torch.tensor([[0.5, -1.0]], requires_grad=True)  # (1, 2)
b = torch.tensor([0.0], requires_grad=True)          # (1,)

print("Formas:")
print("x:", x.shape, "| W:", W.shape, "| b:", b.shape, "| y_true:", y_true.shape)


Formas:
x: torch.Size([1, 2]) | W: torch.Size([1, 2]) | b: torch.Size([1]) | y_true: torch.Size([1, 1])


La fase forward calcula la salida de la neurona.

La fase backward invertirá este flujo: usará cómo cambia la pérdida con respecto a la salida para calcular cómo afecta a los pesos.

In [ ]:
# === 2. Forward: cálculo paso a paso ===

# 1. Cálculo lineal: z = x @ W.T + b
z = x @ W.T + b
print("z (valor pre-activación):", z)

# 2. Aplicar función de activación Sigmoid: a = σ(z)
a = torch.sigmoid(z)
print("a (salida post-Sigmoid):", a)

# 3. Mostrar shapes para confirmar coherencia
print("\nFormas → x:", x.shape, "| W:", W.shape, "| z:", z.shape, "| a:", a.shape)


z (valor pre-activación): tensor([[0.]], grad_fn=<AddBackward0>)
a (salida post-Sigmoid): tensor([[0.5000]], grad_fn=<SigmoidBackward0>)

Formas → x: torch.Size([1, 2]) | W: torch.Size([1, 2]) | z: torch.Size([1, 1]) | a: torch.Size([1, 1])


In [5]:
# === 3. Definir la función de pérdida BCE ===

import torch.nn as nn

# Definimos la función de pérdida adecuada para salidas en [0, 1]
# En este caso usamos BCE debido a que usamos sigmoid como salida
bce_lossfn = nn.BCELoss()

# Calculamos la pérdida entre la predicción 'a' (ya sigmoid) y la etiqueta 'y_true'
loss = bce_lossfn(a, y_true)

#La BCE mide cuánto difiere la probabilidad predicha de la real.

#Es una medida asimétrica: penaliza más cuando la red está segura y se equivoca

#(por ejemplo a≈0.99 cuando y_true=0).

print(f"Pérdida BCE: {loss.item():.4f}")


Pérdida BCE: 0.6931


Calcularemos ahora el backward (backpropagation) y los gradientes



In [6]:
# === 4. Backward: gradientes ===

# 0) Limpiar gradientes previos (por si re-ejecutas la celda)
if W.grad is not None: W.grad.zero_()
if b.grad is not None: b.grad.zero_()

# 1) Gradiente intermedio: dL/da y dL/dz (regla de la cadena)
dL_dz, = torch.autograd.grad(loss, z, retain_graph=True)
dL_da, = torch.autograd.grad(loss, a, retain_graph=True)

print("Gradientes intermedios:")
print("dL/dz:", dL_dz)
print("dL/da:", dL_da)

# 2) Gradientes respecto a parámetros W y b
loss.backward()  # propaga hasta W y b

print("\nGradientes en parámetros:")
print("dL/dW:", W.grad)
print("dL/db:", b.grad)


Gradientes intermedios:
dL/dz: tensor([[-0.5000]])
dL/da: tensor([[-2.]])

Gradientes en parámetros:
dL/dW: tensor([[-1.0000, -0.5000]])
dL/db: tensor([-0.5000])


5. A vigilar como ingeniero

Que los gradientes no se anulen (vanishing gradient, común con Sigmoid).

Que los gradientes no exploten (valores enormes → inestabilidad).

Que las activaciones y pesos estén en escalas razonables.